### Student Information
Name: 劉新正

Student ID: 109065851

GitHub ID: LiuHsinCheng

Kaggle name: liuhsincheng

Kaggle private scoreboard snapshot:

[Snapshot](img/ISA5810-HW2-109065851.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Dec. 15th 11:59 pm, Tuesday)__. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Dec. 18th 11:59 pm, Friday)__. 

### Begin Assignment Here

# 109065851

#!pip3 install pandas
#!pip3 install tensorflow
#!pip3 install tweet-preprocessor
#!pip3 install ktrain




In [ ]:
#!pip3 install pandas 
#!pip3 install tensorflow 
#!pip3 install tweet-preprocessor 
#!pip3 install ktrain

1. Model we use is base on the __BERT model__ and use small implementation __ktrain__
2. Adopt __tweet-processor__ tool to do Twitter sentence preprocess 

In [4]:
# -*- coding: utf-8 -*-
"""
Course # : ISA5810 , Homework #2
Student Name : Liu, Hsin Cheng
Student ID : 109065851
Created on Tue Nov  8 15:35:54 2022

@author: jeffl
"""

import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import preprocessor as p
import re

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: numpy.core.multiarray failed to import

# 1. Data Loading and Do Preprocessing #

## Create functions for text processing ## 

In [ ]:


def preprocess_tweet(text):
    #text = row['text']
    text = p.clean(text)
    return text

# remove non a-z characters

def remove_exchar(text):
    return re.sub(r'[^a-zA-Z]', ' ', text)

# remove html tag from text
def remove_html(text):
  htmltag = re.compile('<.*?>')
  text = re.sub(htmltag, '', text)
  return text




### Do preprocess from aggregated function ### 

In [ ]:
def PreprocessingByTweet(text):
    text = remove_html(text)
    text = preprocess_tweet(text)
    text = remove_exchar(text)
    return text

## Load data set by panadas function ##

In [ ]:
print( tf.__version__ )

# Read JSON file with records orient

datapath = "dataset/tweets_DM.json"
data_ = pd.read_json(datapath, orient='records', lines=True )
output = pd.read_csv('dataset/sampleSubmission.csv')
data_rec = pd.json_normalize(data_.to_dict("records"))
data_rec = data_rec.rename({'_source.tweet.tweet_id': 'tweet_id', '_source.tweet.text': 'text'}, axis=1)
print('data head : ' , data_.head() , '\n', data_rec.iloc[0:5, -1])

data_id = pd.read_csv('dataset/data_identification.csv')
data_emotion = pd.read_csv('dataset/emotion.csv')

__Encode label text to numeric presentation__

In [ ]:
labelencoder = LabelEncoder()
emotion_class_name = data_emotion['emotion'].unique().tolist()
print(emotion_class_name )
data_emotion["encode"] = labelencoder.fit_transform(data_emotion['emotion'])
print(data_emotion.emotion.value_counts())

__Create dictionary of label, encode pair__

In [ ]:
label_dict = dict (zip( data_emotion['encode'],data_emotion['emotion']))

__merge tweets and emotion__

In [ ]:


# using merge function by setting how='outer' to join emotion and identification data set

data_emotion_id = pd.merge(data_id, data_emotion, on='tweet_id', how='outer')
print( data_emotion_id['emotion'].isna().sum() )
# using merge function by setting how='outer' to join emotion and identification data set

data_mix = pd.merge(data_emotion_id, data_rec, on='tweet_id', how='outer')
print( "data_all columns :" , data_mix.columns )





__Keep id, emotion, encode, text column and separate train and target test data__

In [ ]:
data_all = data_mix[['identification', 'tweet_id','emotion','encode','text']]
mask = data_all['identification'] == 'train'


__Separate train and target test data__

In [ ]:

remain_cols = ['tweet_id','emotion','encode','text']
label_cols = ['encode']
train_ = data_all[mask][remain_cols]
test_ = data_all[~mask][remain_cols]

test_ = test_.drop('encode',axis = 1)
train_['ptext'] = train_['text'].apply(PreprocessingByTweet)
print( "data_all columns :" , train_.columns )

In [ ]:

X_, X_test = train_test_split( train_, test_size = 0.15, random_state = 0)

label_ = X_[label_cols]

# 2. Data Sampling #

### Data Size and Label Imbalance ###

Because of the size of data set, I decide to 
1. __cut down the size__
2. solve the __label imbalance__ problem
3. __speed up training__ time

In [ ]:


# under sampling
# find the minimum size within labels

min_class_count = min(train_['encode'].value_counts())

grouped = train_.groupby(['encode'])['ptext'].apply(lambda x: x.sample(min_class_count))
data_under = grouped.reset_index()

# get rid of first row
data_under.drop(['level_1'], axis=1, inplace=True)


### sampling data based on the label size, get evenly on all label  ###

In [ ]:

data_under = data_under.sample(frac=1)
data_under.reset_index(inplace=True, drop=True)
print("Emotion count:", data_under.encode.value_counts())

### Split training and validation data ###

In [ ]:
X_, X_test = train_test_split( data_under, test_size = 0.2, random_state = 0)
#X_, X_test = train_test_split( train_, test_size = 0.2, random_state = 0)

X_train, X_validation = train_test_split( X_, test_size = 0.2, random_state = 0, stratify=X_.encode)

# 3. Ktrain Model Instanciation #

### Create train, test and preproc object for the model ###

In [ ]:


(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_df( train_df = X_train,
                                                                    text_column='ptext',
                                                                    label_columns='encode',
                                                                    val_df = X_validation, 
                                                                    maxlen=150, 
                                                                    ngram_range=2,
                                                                    preprocess_mode='bert')
                                                                    



## Model Description ##


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner( model, train_data=(x_train, y_train), 
                             val_data=(x_test, y_test),
                             batch_size=6)

n = learner.fit_onecycle(2e-5, epochs = 2)
print(n)

learner.validate(val_data=(x_test, y_test), class_names=emotion_class_name)



__get predictor from the model and save to file, in order to use future__

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save("./models/sentiment_analysis")

# Load model and do prediction #

In [ ]:
reloaded_predictor = ktrain.load_predictor('./models/sentiment_analysis')
prediction = reloaded_predictor.predict(test_['text'].values)
test_['predict'] = prediction

## Mapping encode to text label for validation ##

In [ ]:
test_['n']=test_['predict'].str.replace("encode_", "" )
test_['n']=test_['n'].str.replace(".0", "" )
test_['n']=pd.to_numeric(test_['n'], downcast='unsigned')

for ind in test_.index:
    test_['emotion'][ind] = label_dict.get(test_['n'][ind])

test_.rename({'tweet_id': 'id'}, axis=1)
prediction = pd.DataFrame(test_, columns=['id','emotion']).to_csv('sampleSubmission.csv',index=False)
